Your tasks are the following.


---


1. Create and train a FeedForward Neural Network with PyTorch Lightning. Use ModelCheckPoint as callback, and save the best performing model (in terms of MAE) into a ckpt file named *best_model.ckpt*. 50 points


---


2. Save the MAE and the R2 score of your best performing model into a csv file called *mlp_results.csv*. The MAE must be lower and the R2 score must be higher than in linear regression. 20 points.


---


3. Ensemble your neural network model with RandomForestRegressor following the formula
`y_pred_ensemble = 0.5 * y_pred_FeedForwardNet + 0.5 * y_pred_RandomForestRegressor`.
Save the MAE and the R2 score of the ensemble into a csv file called *ensemble_results.csv*.  30 points


---


Besides the Python codes / IPYNB, the CSV files must be saved directly into the git repository you submit.


In [1]:
!pip install pytorch-lightning --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 40.6 MB/s eta 0:00:00


In [2]:
# Dataset
!wget https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv

--2023-10-31 00:16:15--  https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54288 (53K) [text/plain]
Saving to: ‘insurance.csv’

insurance.csv       100%[===================>]  53.02K  --.-KB/s    in 0.002s  

2023-10-31 00:16:15 (26.9 MB/s) - ‘insurance.csv’ saved [54288/54288]



In [3]:
import pandas as pd # pandas for data manipulation
import numpy as np # numpy for linear algebra
import matplotlib.pyplot as plt # matplotlib for plotting
import seaborn as sns # seaborn for plotting

In [4]:
df = pd.read_csv("insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
df = pd.get_dummies(df, drop_first=True) # we have to talk about drop_first -> avoid the dummy variable trap
df

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0,1,0,0
1334,18,31.920,0,2205.98080,0,0,0,0,0
1335,18,36.850,0,1629.83350,0,0,0,1,0
1336,21,25.800,0,2007.94500,0,0,0,0,1


In [6]:
df.dtypes

age                   int64
bmi                 float64
children              int64
charges             float64
sex_male              uint8
smoker_yes            uint8
region_northwest      uint8
region_southeast      uint8
region_southwest      uint8
dtype: object

In [7]:
df.iloc[:, :4] = df.iloc[:, :4].astype('float32')

<ipython-input-7-890d8b6fbeb9>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :4] = df.iloc[:, :4].astype('float32')


In [8]:
df.dtypes

age                 float32
bmi                 float32
children            float32
charges             float32
sex_male              uint8
smoker_yes            uint8
region_northwest      uint8
region_southeast      uint8
region_southwest      uint8
dtype: object

In [9]:
X = df.drop(columns=['charges']).values # features
y = df['charges'].values # label

In [10]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl
import torchmetrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [11]:
from sklearn.preprocessing import StandardScaler

def create_dataloader(X, y, batch_size, shuffle):
  scaler = StandardScaler()
  scaler.fit(X_train)
  X = scaler.transform(X).astype('float32')
  X = torch.from_numpy(X)
  y = y.astype('float32')
  y = torch.from_numpy(y)
  dataset = TensorDataset(X, y)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
  return dataloader

batch_size = 64

train_loader = create_dataloader(X_train, y_train, batch_size, True)
val_loader = create_dataloader(X_val, y_val, batch_size, False)
test_loader = create_dataloader(X_test, y_test, batch_size, False)

In [12]:
from sklearn.metrics import r2_score

class MyFeedForwardNet(pl.LightningModule):
    def __init__(self, input_dim, output_dim, lr):
      super(MyFeedForwardNet, self).__init__()
      self.save_hyperparameters()
      self.lr = lr
      self.layers = nn.Sequential(
           nn.Linear(input_dim, 512),
           nn.ReLU(),
           nn.Dropout(0.2),
           nn.BatchNorm1d(512),  # Batch Normalization
           nn.Linear(512, 256),
           nn.ReLU(),
           nn.Dropout(0.2),
           nn.BatchNorm1d(256),  # Batch Normalization
           nn.Linear(256, 64),
           nn.ReLU(),
           nn.Dropout(0.2),
           nn.BatchNorm1d(64),  # Batch Normalization
           nn.Linear(64, output_dim)
      )
      self.loss_fn = nn.L1Loss()

    # forward propagation
    def forward(self, x):
      return self.layers(x)

    def on_epoch_start(self):
      print(f"Epoch {self.current_epoch} started. Training...")

    # one step of training
    def training_step(self, batch, batch_idx):
      inputs, targets = batch
      outputs = self(inputs).squeeze()
      loss = self.loss_fn(outputs, targets)
      self.log('train_loss', loss)
      return loss

    # one step of validation
    def validation_step(self, batch, batch_idx):
      inputs, targets = batch
      outputs = self(inputs).squeeze()
      loss = self.loss_fn(outputs, targets)
      self.log('val_loss', loss)
      print("val-loss at epoch {} : {}".format(self.current_epoch, loss.item()))
      return loss

    def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=self.lr)

In [13]:
logger = pl.loggers.TensorBoardLogger("logs/", name="heart_disease_logs")

In [14]:
epochs = 120
lr = 0.01
output_dim = 1

# we instantiate our model
model = MyFeedForwardNet(X_train.shape[1], output_dim, lr)

# we use the ModelCheckpoint callback to save the best model
callback = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath = '',
    filename = 'best_model',
    save_top_k=1,
    mode='min'
)

# we use the Trainer class to train our model
trainer = pl.Trainer(
    logger=logger,
    max_epochs=epochs,
    log_every_n_steps=1,
    callbacks=[callback]
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model, train_loader, val_loader) # train the model

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory  exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | layers  | Sequential | 154 K 
1 | loss_fn | L1Loss     | 0     
---------------------------------------
154 K     Trainable params
0         Non-trainable params
154 K     Total params
0.616     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


val-loss at epoch 0 : 14276.3251953125
val-loss at epoch 0 : 13872.4306640625


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

val-loss at epoch 118 : 1764.021484375
val-loss at epoch 118 : 2745.452880859375


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=120` reached.


val-loss at epoch 119 : 1929.66455078125
val-loss at epoch 119 : 2997.396728515625


In [16]:
from sklearn.metrics import mean_absolute_error, r2_score

# load best model
best_model = MyFeedForwardNet.load_from_checkpoint(callback.best_model_path)

predicts_list = []
targets_list = []
for batch in test_loader:
    inputs, targets = batch
    predicts_batch = best_model(inputs).squeeze().tolist()
    targets_batch = targets.tolist()
    predicts_list.extend(predicts_batch)
    targets_list.extend(targets_batch)

# calculate MAE and R2
mae = mean_absolute_error(targets_list, predicts_list)
r2 = r2_score(targets_list, predicts_list)

# save MAE and R2 to .csv
results = pd.DataFrame({
    'MAE': [mae],
    'R2 Score': [r2]
})
results.to_csv('mlp_results.csv', index=False)

print(f'MAE: {mae}')
print(f'R2 Score: {r2}')

MAE: 2643.1366767313943
R2 Score: 0.8413943335525023


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

y_pred_ensemble = []
scaler = StandardScaler()
scaler.fit(X_train)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(scaler.transform(X_train), y_train)
y_pred_RandomForestRegressor = rf_model.predict(scaler.transform(X_test))
y_pred_ensemble = [0.5 * nn_pred + 0.5 * rf_pred for nn_pred, rf_pred in zip(predicts_list, y_pred_RandomForestRegressor)]

# calculate MAE and R2
mae = mean_absolute_error(targets_list, y_pred_ensemble)
r2 = r2_score(targets_list, y_pred_ensemble)

# save MAE and R2 to .csv
results = pd.DataFrame({
    'MAE': [mae],
    'R2 Score': [r2]
})
results.to_csv('ensemble_results.csv', index=False)

print(f'MAE: {mae}')
print(f'R2 Score: {r2}')

MAE: 2444.2760507088274
R2 Score: 0.8711537194871536
